# Qwen2-7b-it
For your reference, I'll share an agent created with Qwen2-7b-it which has been performing well on the [LLM Leaderboard](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard).

In [ ]:
%%bash
mkdir -p /kaggle/working/submission
pip install bitsandbytes accelerate
pip install -U transformers

In [ ]:
%%writefile -a submission/main.py

import os
import torch
import random
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, pipeline

torch.backends.cuda.enable_mem_efficient_sdp(False)

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "1")
else:
    model_id = "/kaggle/input/qwen2/transformers/qwen2-7b-instruct/1"


tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    quantization_config=quantization_config,
    torch_dtype=torch.float16,
)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

def parse_response(response):
    match = re.search(".+?\?", response.replace('*', ''))
    if match is None:
        question = 'Does the keyword begins with th letter "s"?'
    else:
        question = match.group()
    return question

def generate_answer(chat_template):
    output = pipe(
        chat_template,
        max_new_tokens=32,
        do_sample=False,
#         temperature=0.01,
#         top_p=0.1,
#         top_k=1,
        return_full_text=False,
    )[0]["generated_text"]
    output = re.sub('<end_of_turn>', '', output)
    return output


class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        assert mode in ["asking", "guessing", "answering"], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            output = self.asker(obs)
        if mode == "answering":
            output = self.answerer(obs)
            if "yes" in output.lower() or "Yes" in output.lower():
                output = "yes"
            elif "no" in output.lower() or "No" in output.lower():
                output = "no"
            else:
                output = "no"
        if mode == "guessing":
            output = self.asker(obs)
        return output

    def asker(self, obs):
        sys_prompt = """
        You are an AI assistant designed to play the 20 Questions game. 
        In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner.
        The keyword is a specific "thing" or "place".
        """
        if obs.turnType =="ask":
            ask_prompt = sys_prompt + """
            Let's play 20 Questions. You are playing the role of the Questioner.
            Please ask a yes-or-no question.
            to help you, here's an example of how it should work:
            examle: keyword is "Morocco":
            <assistant: is it a city?
            user: no
            assistant: is it a country?
            user: yes
            assistant: is it in africa?
            user: yes
            assistant: is it a country name starting by m ?
            user: yes
            assistant: is it Morocco?
            user: yes.>
            """

            chat_template = f"""<start_of_turn>system\n{ask_prompt}\n"""

            if len(obs.questions)>=1:
                chat_template += "Here's the conversation history so far:\n"
                chat_template += "<\n"
                for q, a in zip(obs.questions, obs.answers):
                    chat_template += f"assistant:\n{q}\n"
                    chat_template += f"user:\n{a}\n"
                chat_template += ">\n"
                    
            chat_template += """
            the user has chosen the word, ask your first question!
            please be short and not verbose, give only one question, no extra word!<end_of_turn>
            <start_of_turn>assistant\n
            """
#             print(chat_template)
            output = generate_answer(chat_template) 
            output = parse_response(output)
                    
        elif obs.turnType == "guess":
            conv = ""
            for q, a in zip(obs.questions, obs.answers):
                conv += f"""question:\n{q}\nanswer:\n{a}\n"""
            guess_prompt =  sys_prompt + f"""
            Your role is to predict keywords based on questions and answers.
            to help you, here's an example of how it should work:
            examle:
            <question: is it a city?
            answer: no
            question: is it a country?
            answer: yes
            question: is it in africa?
            answer: yes
            question: is it a country name starting by m ?
            answer: yes
            you: Morocco>
            
            so far, the current state of the game is as following:\n{conv}
            based on the conversation, can you guess the word, please give only the word, no verbosity around<end_of_turn>
            """
            chat_template = f"""<start_of_turn>system\n{guess_prompt}\n"""
            chat_template += "<start_of_turn>you:\n"
#             print(chat_template)
            output = generate_answer(chat_template)        
        return output
    
    def answerer(self, obs):
        ask_prompt = f"""
        You are an AI assistant designed to play the 20 Questions game. 
        In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner.
        to help you, here's an example of how it should work:
        examle: keyword is "Morocco"
        <user: is it a place?
        you: yes
        user: is it in europe?
        you: no
        user: is it in africa?
        you: yes
        user: is it a country name starting by m ?
        you: yes
        user: is it Morocco?
        you: yes.>
        About "{obs.keyword}", please answer the following question with "yes" or "no":
        {obs.questions[-1]}
        """
        chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
        chat_template += "<start_of_turn>Assistant\n"
        output = generate_answer(chat_template)
        return output

robot = Robot()

def agent(obs, cfg):
    
    if obs.turnType =="ask":
        response = robot.on(mode = "asking", obs = obs)
        
    elif obs.turnType =="guess":
        response = robot.on(mode = "guessing", obs = obs)
        
    elif obs.turnType =="answer":
        response = robot.on(mode = "answering", obs = obs)
        
    if response == None or len(response)<=1:
        response = "yes"
        
    return response

In [ ]:
!apt install pigz pv > /dev/null

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/input/qwen2/transformers/qwen2-7b-instruct . -C /kaggle/working/submission .

In [ ]:
# # to see what's inside tar.gz file

# import tarfile
# tar = tarfile.open("/kaggle/working/submission.tar.gz")
# for file in tar.getmembers():
#     print(file.name)

# Simulation
If you want to simulate your agent before submitting it, uncomment the following and run it.

In [ ]:
# !wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py

In [ ]:
# import json
# import pandas as pd
# from submission.main import agent
# from keywords_local import KEYWORDS_JSON

# class Observation:
#     def __init__(self):
#         self.step = 0
#         self.role = "guesser"
#         self.turnType = "ask"
#         self.keyword = "Japan"
#         self.category = "country"
#         self.questions = []
#         self.answers = []
#         self.guesses = []
        
# def create_keyword_df(KEYWORDS_JSON):
#     json_data = json.loads(KEYWORDS_JSON)

#     keyword_list = []
#     category_list = []
#     alts_list = []

#     for i in range(len(json_data)):
#         for j in range(len(json_data[i]['words'])):
#             keyword = json_data[i]['words'][j]['keyword']
#             keyword_list.append(keyword)
#             category_list.append(json_data[i]['category'])
#             alts_list.append(json_data[i]['words'][j]['alts'])

#     data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])
#     data_pd['keyword'] = keyword_list
#     data_pd['category'] = category_list
#     data_pd['alts'] = alts_list
    
#     return data_pd
    
# keywords_df = create_keyword_df(KEYWORDS_JSON)

In [ ]:
# obs = Observation()
# cfg = "_"

# sample_df = keywords_df.sample()
# obs.keyword = sample_df["keyword"].values[0]
# obs.category = sample_df["category"].values[0]
# alts_list = sample_df["alts"].values[0]
# alts_list.append(obs.keyword)

# print(f"keyword:{obs.keyword}")

# for round in range(20):
#     obs.step = round+1
    
#     obs.role = "guesser"
#     obs.turnType = "ask"
#     question = agent(obs, cfg)
#     obs.questions.append(question)
    
#     obs.role = "answerer"
#     obs.turnType = "answer"
#     answer = agent(obs, cfg)
#     obs.answers.append(answer)
    
#     obs.role = "guesser"
#     obs.turnType = "guess"
#     guess = agent(obs, cfg)
#     obs.guesses.append(guess)
    
#     print(f"round: {round+1}")
#     print(f"question: {question}")
#     print(f"answer: {answer}")
#     print(f"guess: {guess}")
    
#     if guess in alts_list:
#         print("Win!!")
#         break